In [27]:
def loadWeatherData(fileName, getSpatialData = True):
    retrievedData = []
    
    with open(fileName) as data:
        loadedData = data.readlines()

        #TODO: Load weather data
        uncleanedWeatherData = loadedData[99:200]
        weatherData = [i.split(',') for i in uncleanedWeatherData[:]]
        for lines in weatherData:
            for i, values in enumerate(lines):
                chars = [char for char in list(values) if i not in (' ', '\n')]
                lines[i] = ''.join(chars)
                
                #Error handling required to prevent unexpected EOF while parsing         
                try:
                    lines[i] = eval(lines[i])
                except:
                    pass
                    
            
        if getSpatialData == True:
            uncleanedSpatialData = loadedData[5:55]
            
            #Splitting and cleaning of metadata lines
            splitSpatialData = [i.split() for i in uncleanedSpatialData[:]]
                       
            spatialData = list(map(lambda values: values[1:], splitSpatialData))
            for lines in spatialData:
                lines[0] = lines[0][:-1]
    return weatherData, spatialData

SyntaxError: future feature eval is not defined (<ipython-input-27-2522e9f26c8d>, line 1)

In [26]:
data = loadWeatherData(r'D:\git\pandas-bokeh\data\KNMI_20161227.txt')

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [20]:
print data[0][10]

[209, 20010209, 22, 40, 48, 100, 1, 20, 20, 130, 3, '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     ', '     \n']
